# Coleta de Dados da Rede Algorand

    - Importando bibliotecas:

In [2]:
from matplotlib import pyplot as plt
from lib2to3.pytree import convert
from ast import literal_eval
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
from ctypes import *
import pandas as pd
import numpy as np
import requests
import struct
import json
import time
import math

--------------------------------------------------------------------------------------

### Coleta de Transações:

    - Classificação dos atributos:


- id:              Hash da transação;
- current-round:   Numero do bloco atual;
- confirmed-round: Numero do bloco;
- round-time:      Timestamp do bloco;
- type:            Tipo da transação;
- fee:             Taxas da transação -> fee = MAX(current_fee_per_byte * len(tnx_in_bytes), min_fee);

In [10]:
#? Inicializando objetos path;
file_obj_1 = Path('../data/transactions/algo_transactions_8.csv')
file_obj_2 = Path('../data/transactions/atual_block.txt')

#? Inicialização do dataframe;
if file_obj_1.is_file():
    df = pd.read_csv('../data/transactions/algo_transactions_8.csv')
else:
    df = pd.DataFrame()

#? Inicialização do número do último bloco coletado;
if file_obj_2.is_file():
    with open("../data/transactions/atual_block.txt", "r") as arq:
        number = arq.read()
else:
    number = 0

In [ ]:
for i in tqdm(range(300)):
    round = str(number)
    url = 'https://algoindexer.algoexplorerapi.io/v2/transactions?limit=8000&min-round=' + round

    block = requests.get(url)
    block = block.json()
    
    tx = block['transactions']
    tx = pd.DataFrame(tx)

    if df.shape[0] > 0:
        df = pd.concat([df, tx])
        number = df['confirmed-round'].iloc[-1]
    else:
        df = tx
        number = df['confirmed-round'].iloc[-1]

with open('../data/transactions/atual_block.txt', 'w') as arquivo:
        arquivo.write(str(number))

print('Número de transações: ' + str(df.shape[0]))
print('Bloco atual: ' + str(number))

In [13]:
#? Salvar dataframe com transações;
df.to_csv('../data/transactions/algo_transactions_8.csv', index=False)

--------------------------------------------------------------------------------------

### Coleta de Contas:

* Necessário realizar a extração de endereços no arquivo './process.ipynb';

In [17]:
address = pd.read_csv('../data/accounts/address.csv')
address = pd.DataFrame(address)

address_list = []
amount_list = []
amount_without_pending_rewards_list = []
created_at_round_list = []
deleted_list = []       
pending_rewards_list = []
reward_base_list = []                                        
rewards_list = []                                                                
round_list = []                                                                   
sig_type_list = []                                                               
status_list = []                                                               
total_apps_opted_in_list = []                                                        
total_assets_opted_in_list = []                                                        
total_created_apps_list = []                                                           
total_created_assets_list = []                                                        

In [ ]:
import csv


for i in tqdm(range(address.shape[0])):
    url = 'https://algoindexer.algoexplorerapi.io/v2/accounts/' + str(address.at[i, 'address'])

    acc = requests.get(url)
    acc = acc.json()
    acc = pd.DataFrame(acc)
    acc = acc['account']

    address_list.append(acc['address'])
    amount_list.append(acc['amount'])
    amount_without_pending_rewards_list.append(acc['amount-without-pending-rewards'])
    created_at_round_list.append(acc['created-at-round'])
    deleted_list.append(acc['deleted'])
    pending_rewards_list.append(acc['pending-rewards'])
    reward_base_list.append(acc['reward-base'])             
    rewards_list.append(acc['rewards'])                                     
    round_list.append(acc['round'])                                          
    sig_type_list.append(acc['sig-type'])                                           
    status_list.append(acc['status'])                                      
    total_apps_opted_in_list.append(acc['total-apps-opted-in'])                                              
    total_assets_opted_in_list.append(acc['total-assets-opted-in'])                                   
    total_created_apps_list.append(acc['total-created-apps'])                                 
    total_created_assets_list.append(acc['total-created-assets'])

accounts = pd.DataFrame({'address': address_list, 'amount': amount_list, 'amount-without-pending-rewards': amount_without_pending_rewards_list,
'created-at-round': created_at_round_list, 'deleted': deleted_list, 'pending-rewards': pending_rewards_list, 'reward-base': reward_base_list,
'rewards': rewards_list, 'round': round_list, 'sig-type': sig_type_list, 'status': status_list, 'total-apps-opted-in': total_apps_opted_in_list,
'total-assets-opted-in': total_assets_opted_in_list, 'total-created-apps': total_created_apps_list, 'total-created-assets': total_created_assets_list})

accounts.to_csv('../data/accounts/algo_accounts.csv', index=False)

In [23]:


url = 'https://algoindexer.algoexplorerapi.io/v2/accounts/3P3CHL4M5JTDJKEL3ARLUZRXY23BWWPX6SZDC2NIBALL3SKE7JBIFOVOAY'

acc = requests.get(url)
print(type(acc))
acc = acc.json()
#acc = pd.DataFrame(acc)
#acc = acc['account']

print(acc)

<class 'requests.models.Response'>
{'account': {'address': '3P3CHL4M5JTDJKEL3ARLUZRXY23BWWPX6SZDC2NIBALL3SKE7JBIFOVOAY', 'amount': 1103323927, 'amount-without-pending-rewards': 1058180125, 'created-at-round': 68203, 'deleted': False, 'pending-rewards': 45143802, 'reward-base': 175619, 'rewards': 203564678, 'round': 23031400, 'sig-type': 'sig', 'status': 'Offline', 'total-apps-opted-in': 0, 'total-assets-opted-in': 0, 'total-created-apps': 0, 'total-created-assets': 0}, 'current-round': 23031400}
